In [ ]:
import requests
import pandas as pd
import numpy as np
import datetime
from yahooquery import Ticker
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib as plt
import math
from pandas.tseries.offsets import DateOffset
from datetime import datetime

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

import warnings

warnings.filterwarnings("ignore")

In [ ]:
def get_n_pages_given_a_soup(soup):
    num_of_results_raw_html = soup.find('span', class_='Mstart(15px) Fw(500) Fz(s)').get_text()
    num_of_results_split = num_of_results_raw_html.split(' ')
    n_results = int(num_of_results_split[-2])
    n_pages = math.ceil(n_results/100)
    return n_pages

def get_symbols_given_a_soup(soup):
    symbols = []
    symbols_a_html = soup.find_all('a', class_='Fw(600) C($linkColor)')
    for symbol in symbols_a_html:
        symbols.append(symbol.get_text())
    return symbols

In [ ]:
#https://stackoverflow.com/questions/45448994/wait-page-to-load-before-getting-data-with-requests-get-in-python-3

from bs4 import BeautifulSoup
from selenium import webdriver
import sys  

user_agent = 'Mozilla/5.0 (X11; Linux x86_64 AppleWebKit/537.36 (KHTML, like GECKO) Chrome/33.0.1750.517 Safari/537.36'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('user-agent={0}'.format(user_agent))

browser = webdriver.Chrome(options=options, executable_path='/Users/andrew/Documents/workspace/earnings-surprise-modelling/notebooks/chromedriver/chromedriver')
date = '2023-06-12'
url = f"https://finance.yahoo.com/calendar/earnings?day={date}"
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")

symbols = []
n_pages = get_n_pages_given_a_soup(soup)
symbols.extend(get_symbols_given_a_soup(soup))

if n_pages > 1:
    for i in range(1,n_pages):
        i_100 = i*100
        next_page_url = f'https://finance.yahoo.com/calendar/earnings?day={date}&offset=i_100&size=i_100'
        browser.get(url)
        html = browser.page_source
        soup = BeautifulSoup(html, features="html.parser")
        symbols.extend(get_symbols_given_a_soup(soup))

browser.quit()

symbols = list(set(symbols))

In [ ]:
print(symbols)